<a href="https://colab.research.google.com/github/saiku122/AIJobcolle/blob/master/MachineLearning/python/%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92%E6%9C%80%E7%B5%82%E8%A9%A6%E9%A8%93.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/saiku122/AIJobcolle.git

Cloning into 'AIJobcolle'...
remote: Enumerating objects: 455, done.
remote: Counting objects: 100% (455/455), done.
remote: Compressing objects: 100% (334/334), done.
remote: Total 455 (delta 189), reused 215 (delta 68), pack-reused 0
Receiving objects: 100% (455/455), 12.07 MiB | 11.27 MiB/s, done.
Resolving deltas: 100% (189/189), done.


In [2]:
cd /content/AIJobcolle/MachineLearning/python

/content/AIJobcolle/MachineLearning/python


In [9]:
from IPython.core.display import display
import pandas as pd

df = pd.read_csv('./data/av_loan_u6lujuX_CVtuZ9i.csv',header=0)
X=df.iloc[:,1:-1]
y=df.iloc[:,-1]

class_mapping={'N':1,'Y':0}
y=y.map(class_mapping)

display(X.join(y).head())

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,0
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,1
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,0
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,0
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,0


In [10]:
#カテゴリ変数
categoru_columns=['Gender','Married','Dependents','Education','Self_Employed','Property_Area']

In [11]:
y.isnull().sum()

0

In [12]:
X_ohe = pd.get_dummies(X,dummy_na=True,columns=categoru_columns)

In [13]:
display(X_ohe.head())

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Gender_nan,Married_No,Married_Yes,Married_nan,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Dependents_nan,Education_Graduate,Education_Not Graduate,Education_nan,Self_Employed_No,Self_Employed_Yes,Self_Employed_nan,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Property_Area_nan
0,5849,0.0,NaN,360.0,1.0,0,1,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0
1,4583,1508.0,128.0,360.0,1.0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0
2,3000,0.0,66.0,360.0,1.0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0
3,2583,2358.0,120.0,360.0,1.0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0
4,6000,0.0,141.0,360.0,1.0,0,1,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0


In [14]:
from sklearn.impute import SimpleImputer

X_fin=SimpleImputer().fit_transform(X_ohe)

In [30]:
#ホールドアウト法適用
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_fin,
                                              y,
                                              test_size=0.20,
                                              random_state=1)

**ソースコード2**

In [31]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [32]:
lr=Pipeline([
             ('scl',StandardScaler()),
             ('est',LogisticRegression())
             ])
rf1=RandomForestClassifier()
rf2=Pipeline([
             ('scl',StandardScaler()),
             ('pca',PCA()),
             ('est',RandomForestClassifier())
             ])

In [33]:
from sklearn.metrics import f1_score

clfs = {
    'logistic':lr,
    'rf1':rf1,
    'rf2':rf2
}
display(X_train)
for clf_name, clf in clfs.items():
  clf.fit(X_train, y_train)
  print(clf_name,':',f1_score(y_test,clf.predict(X_test)))


array([[4.400e+03, 0.000e+00, 1.270e+02, ..., 1.000e+00, 0.000e+00,
        0.000e+00],
       [3.583e+03, 0.000e+00, 9.600e+01, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       [4.333e+03, 2.451e+03, 1.100e+02, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       ...,
       [3.500e+03, 0.000e+00, 8.100e+01, ..., 1.000e+00, 0.000e+00,
        0.000e+00],
       [5.500e+03, 1.260e+03, 1.700e+02, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [3.667e+03, 1.459e+03, 1.440e+02, ..., 1.000e+00, 0.000e+00,
        0.000e+00]])

logistic : 0.5714285714285713
rf1 : 0.5245901639344261
rf2 : 0.4918032786885246


In [36]:
lr_pg = {
    'est__penalty':('l1','l2'),
    'est__c':[0.1,1.0,10.0]
}

In [43]:
from sklearn.model_selection import GridSearchCV
lr_gs = GridSearchCV(lr,scoring='f1_score',param_grid=lr_pg)
lr_gs.fit(X_fin,y)

ValueError: ignored